In [1]:
!pip install pgmpy

  Using cached torch-2.9.0-cp312-none-macosx_11_0_arm64.whl.metadata (30 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.2 MB/s eta 0:00:00
Using cached opt_einsum-3.4.0-py3-none-any.whl (71 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 9.1 MB/s eta 0:00:00
Using cached torch-2.9.0-cp312-none-macosx_11_0_arm64.whl (74.5 MB)
Using cached sympy-1.14.0-py3-none-any.whl (6.3 MB)
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.2
    Uninstalling sympy-1.13.2:
      Successfully uninstalled sympy-1.13.2


In [3]:
import pandas as pd
import numpy as np
from pgmpy.models import BayesianNetwork, DiscreteBayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

In [5]:
model = DiscreteBayesianNetwork([('Prize', 'Host'), ('Choice', 'Host')])

In [7]:
cpd_prize = TabularCPD(variable='Prize', variable_card=3,
                      values=[[1/3], [1/3], [1/3]],
                      state_names={'Prize': [1, 2, 3]})

In [9]:
cpd_choice = TabularCPD(variable='Choice', variable_card=3,
                       values=[[1/3], [1/3], [1/3]],
                       state_names={'Choice': [1, 2, 3]})

In [11]:
host_values = np.zeros((3, 9))

In [13]:
host_doors = [1, 2, 3]
for i, prize in enumerate(host_doors):
   for j, choice in enumerate(host_doors):
       valid = [door for door in host_doors if door != prize and door != choice]
       for v in valid:
           host_values[v-1][3*i + j] = 1 / len(valid)

In [15]:
cpd_host = TabularCPD(variable='Host', variable_card=3,
                     values=host_values,
                     evidence=['Prize', 'Choice'],
                     evidence_card=[3, 3],
                     state_names={'Host': [1, 2, 3], 'Prize': [1, 2, 3], 'Choice': [1, 2, 3]})

In [17]:
model.add_cpds(cpd_prize, cpd_choice, cpd_host)
model.check_model()

True

In [19]:
inference = VariableElimination(model)

In [21]:
query_result = inference.query(variables=['Prize'],
                              evidence={'Choice': 1, 'Host': 3})

In [23]:
print("P(Prize | Choice=1, Host=3):")
print(query_result)

print("\nBest strategy: switch to the door with highest probability.")

P(Prize | Choice=1, Host=3):
+----------+--------------+
| Prize    |   phi(Prize) |
+==========+==============+
| Prize(1) |       0.3333 |
+----------+--------------+
| Prize(2) |       0.6667 |
+----------+--------------+
| Prize(3) |       0.0000 |
+----------+--------------+

Best strategy: switch to the door with highest probability.
